In [12]:
import mlflow
import numpy as np

def get_metrics_and_params(run_id):
    client = mlflow.tracking.MlflowClient(tracking_uri='http://127.0.0.1:5000/')
    run = client.get_run(run_id)
    metrics = run.data.metrics
    params = run.data.params
    return metrics, params

def get_mean_and_std(val1, val2, val3):
    mean = (val1 + val2 + val3) / 3
    std = np.sqrt(((val1 - mean)**2 + (val2 - mean)**2 + (val3 - mean)**2) / 3)
    return round(mean,2), round(std,2)

def get_metrics_overview(retrained_metrics_1, retrained_metrics_2, retrained_metrics_3, unlearned_metrics_1, unlearned_metrics_2, unlearned_metrics_3, percentage=True):
    if 'MIA_prob' not in retrained_metrics_1:
        retrained_metrics_1['MIA_prob'] = retrained_metrics_1['membership_attack_prob']
    if percentage:
        for met in ["MIA_prob", "acc_forget", "acc_retain"]:
            retrained_metrics_1[met] = round(retrained_metrics_1[met]/100,2)
            retrained_metrics_2[met] = round(retrained_metrics_2[met]/100,2)
            retrained_metrics_3[met] = round(retrained_metrics_3[met]/100,2)


    r_mia_mean, r_mia_std = get_mean_and_std(retrained_metrics_1['MIA_prob'], retrained_metrics_2['MIA_prob'], retrained_metrics_3['MIA_prob'])
    r_acc_forget_mean, r_acc_forget_std = get_mean_and_std(retrained_metrics_1['acc_forget'], retrained_metrics_2['acc_forget'], retrained_metrics_3['acc_forget'])
    r_acc_retain_mean, r_acc_retain_std = get_mean_and_std(retrained_metrics_1['acc_retain'], retrained_metrics_2['acc_retain'], retrained_metrics_3['acc_retain'])
    r_t_mean, r_t_std = get_mean_and_std(retrained_metrics_1['t'], retrained_metrics_2['t'], retrained_metrics_3['t'])
    r_l2_mean, r_l2_std = get_mean_and_std(retrained_metrics_1['l2'], retrained_metrics_2['l2'], retrained_metrics_3['l2'])

    u_mia_mean, u_mia_std = get_mean_and_std(unlearned_metrics_1['MIA_prob'], unlearned_metrics_2['MIA_prob'], unlearned_metrics_3['MIA_prob'])
    u_acc_forget_mean, u_acc_forget_std = get_mean_and_std(unlearned_metrics_1['acc_forget'], unlearned_metrics_2['acc_forget'], unlearned_metrics_3['acc_forget'])
    u_acc_retain_mean, u_acc_retain_std = get_mean_and_std(unlearned_metrics_1['acc_retain'], unlearned_metrics_2['acc_retain'], unlearned_metrics_3['acc_retain'])
    u_t_mean, u_t_std = get_mean_and_std(unlearned_metrics_1['t'], unlearned_metrics_2['t'], unlearned_metrics_3['t'])
    u_l2_mean, u_l2_std = get_mean_and_std(unlearned_metrics_1['l2'], unlearned_metrics_2['l2'], unlearned_metrics_3['l2'])
    u_ve_mean, u_ve_std = get_mean_and_std(unlearned_metrics_1['VE'], unlearned_metrics_2['VE'], unlearned_metrics_3['VE'])

    diff_acc_forget_mean, diff_acc_forget_std = get_mean_and_std(abs(retrained_metrics_1['acc_forget'] - unlearned_metrics_1['acc_forget']), abs(retrained_metrics_2['acc_forget'] - unlearned_metrics_2['acc_forget']), abs(retrained_metrics_3['acc_forget'] - unlearned_metrics_3['acc_forget']))
    diff_acc_retain_mean, diff_acc_retain_std = get_mean_and_std(abs(retrained_metrics_1['acc_retain'] - unlearned_metrics_1['acc_retain']), abs(retrained_metrics_2['acc_retain'] - unlearned_metrics_2['acc_retain']), abs(retrained_metrics_3['acc_retain'] - unlearned_metrics_3['acc_retain']))

    retrained_metrics_overview = {"MIA": f"{r_mia_mean} ± {r_mia_std}", "acc_forget": f"{r_acc_forget_mean} ± {r_acc_forget_std}", "acc_retain": f"{r_acc_retain_mean} ± {r_acc_retain_std}", "t": f"{r_t_mean} ± {r_t_std}", "l2": f"{r_l2_mean} ± {r_l2_std}"}
    unlearned_metrics_overview = {"MIA": f"{u_mia_mean} ± {u_mia_std}", "acc_forget": f"{u_acc_forget_mean} ± {u_acc_forget_std} ({diff_acc_forget_mean} ± {diff_acc_forget_std})", "acc_retain": f"{u_acc_retain_mean} ± {u_acc_retain_std} ({diff_acc_retain_mean} ± {diff_acc_retain_std})", "t": f"{u_t_mean} ± {u_t_std}", "l2": f"{u_l2_mean} ± {u_l2_std}", "ve": f"{u_ve_mean} ± {u_ve_std}"}
    return retrained_metrics_overview, unlearned_metrics_overview

# Instance-wise Unlearning
Description: Forget specific datapoints across all the classes.

## CIFAR-10 (ResNet18)

In [13]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("4933a16e6e594a849a80a1aebb0132a8")
retrained_metrics2, retrained_params2 = get_metrics_and_params("115c419853bd4ca5b7812bb0a39411bb")
retrained_metrics3, retrained_params3 = get_metrics_and_params("6f1f93b6b21c44c5b232f92819931885")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("8b8049e1778747b983b1ec4f8cc7847e")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("59f2c21a24ea4ceaaf93b32e27b45400")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("d6b39048de914a3682419ba542aad400")

# Hyper-parameters of unlearned models
print(f"Hyperparams: {unlearned_params1}, {unlearned_params2}, {unlearned_params3}")

cifar10_resnet_retrained_metrics, cifar10_resnet_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3)
print(f"Retrained: {cifar10_resnet_retrained_metrics}")
print(f"Unlearned: {cifar10_resnet_unlearned_metrics}")

Hyperparams: {'reference_run_name': 'retrained', 'reference_run_id': '4933a16e6e594a849a80a1aebb0132a8', 'seed': '3407', 'cudnn': 'slow', 'dataset': 'cifar-10', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'lr': '0.0001', 'wd': '0.0005', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0.0005\n)', 'tempScheduler': 'exponential', 'minTemp': '1.1', 'maxTemp': '16.0', 'probTransformer': 'gumbel-softmax', 'datetime': '09-09-16-53', 'epochs': '10', 'method': 'our', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'model': 'resnet18', 'batch_size': '128', 'beta': '0'}, {'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'probTransformer': 'gumbel-softmax', 'beta': '0', 'reference_run_name': 'retrained', 'seed': '13', 'cudnn': 'slow', 'model': 'resnet18', 'method': 'ou

## CIFAR-100 (ResNet18)

In [14]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("d2d5d47fee764f8aa3ef70f464e8fc71")
retrained_metrics2, retrained_params2 = get_metrics_and_params("807e0d22e0714b6e85f08d2f4c40c1f4")
retrained_metrics3, retrained_params3 = get_metrics_and_params("080fcf72b43a40c1b44f6a4d472b4b26")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("fec0002f10214d4fa849d3508d3fc18e")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("61aaf4e9a4134d268b10e313fafe7a08")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("a82abae8634f4246b060c89c29b73cee")

# Hyper-parameters of unlearned models
print(unlearned_params1, unlearned_params2, unlearned_params3)

cifar100_resnet_retrained_metrics, cifar100_resnet_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3)
print(f"Retrained: {cifar100_resnet_retrained_metrics}")
print(f"Unlearned: {cifar100_resnet_unlearned_metrics}")

{'seed': '3407', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'maxTemp': '4.0', 'beta': '0', 'cudnn': 'slow', 'dataset': 'cifar-100', 'model': 'resnet18', 'epochs': '10', 'method': 'our', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'lr': '0.0001', 'wd': '0.0005', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0.0005\n)', 'tempScheduler': 'exponential', 'datetime': '09-09-17-16', 'reference_run_name': 'retrained', 'reference_run_id': 'd2d5d47fee764f8aa3ef70f464e8fc71', 'minTemp': '1.1', 'probTransformer': 'gumbel-softmax', 'batch_size': '128'} {'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'wd': '0.0005', 'tempScheduler': 'exponential', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'lr': '0.0001', 'optimizer': 'Adam (\nParameter Group 0\n    am

## MUFAC (ResNet18)

In [15]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("7fbc281913c74e6aa1eef899758b9ff2")
retrained_metrics2, retrained_params2 = get_metrics_and_params("cb0ce77530b84dc193593a6d340ae978")
retrained_metrics3, retrained_params3 = get_metrics_and_params("28a660ce59e5477da488052c62d717cc")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("a60f775a1f744dfeaae939493644bf32")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("a842b31ae3e14bd5a33bccda833118bb")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("b7e788aee6d54ead88186c3c1388b454")

# Hyper-parameters of unlearned models
print(unlearned_params1, unlearned_params2, unlearned_params3)

mufac_resnet_retrained_metrics, mufac_resnet_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3, percentage=False)
print(f"Retrained: {mufac_resnet_retrained_metrics}")
print(f"Unlearned: {mufac_resnet_unlearned_metrics}")

{'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'lr': '0.0001', 'minTemp': '1.1', 'reference_run_name': 'retrained', 'reference_run_id': '7fbc281913c74e6aa1eef899758b9ff2', 'batch_size': '128', 'epochs': '10', 'method': 'our', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'datetime': '09-09-18-05', 'seed': '3407', 'cudnn': 'slow', 'dataset': 'mufac', 'model': 'resnet18', 'wd': '0.0005', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0.0005\n)', 'tempScheduler': 'exponential', 'maxTemp': '1024.0', 'beta': '0', 'probTransformer': 'gumbel-softmax'} {'reference_run_id': 'cb0ce77530b84dc193593a6d340ae978', 'reference_run_name': 'retrained', 'seed': '13', 'model': 'resnet18', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: 

## IageNet (ResNet18)

## CIFAR-10 (ViT)

In [18]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("1f63dc6939464f389da7a9582c31e92c")
retrained_metrics2, retrained_params2 = get_metrics_and_params("749d4bea8d0e478581608b739f9cfee4")
retrained_metrics3, retrained_params3 = get_metrics_and_params("4a5b40adcd8948498b69a0b860f8f1d6")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("9fa1ee7a4f5f4a60bed7a65469d92129")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("136b46bdc0074668914a543228193ee0")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("939b37f91cca4cd7bb6c480735e3b48e")

# Hyper-parameters of unlearned models
print(unlearned_params1, unlearned_params2, unlearned_params3)

cifar10_vit_retrained_metrics, cifar10_vit_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3, percentage=True)
print(f"Retrained: {cifar10_vit_retrained_metrics}")
print(f"Unlearned: {cifar10_vit_unlearned_metrics}")

{'reference_run_id': '1f63dc6939464f389da7a9582c31e92c', 'cudnn': 'slow', 'dataset': 'cifar-10', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 1e-06\n    maximize: False\n    weight_decay: 0.0005\n)', 'lr': '1e-06', 'wd': '0.0005', 'tempScheduler': 'exponential', 'minTemp': '1.1', 'datetime': '09-09-21-29', 'seed': '3407', 'model': 'vit', 'reference_run_name': 'retrained', 'batch_size': '32', 'epochs': '3', 'method': 'our', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'maxTemp': '2.0', 'probTransformer': 'gumbel-softmax', 'beta': '0'} {'reference_run_name': 'retrained', 'branch_name': 'annealing', 'beta': '0', 'method': 'our', 'datetime': '09-09-21-15', 'batch_size': '32', 'epochs': '3', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'Dr_subset_size': '

## CIFAR-100 (ViT)

In [19]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("7e1d79e5608e4505bf94b34312394f40")
retrained_metrics2, retrained_params2 = get_metrics_and_params("222b19b4fb4a43a4b39b815cefd3913e")
retrained_metrics3, retrained_params3 = get_metrics_and_params("33b0b032d8144814acfc6bf8e9894ea5")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("decf0822275242c598c306178a8c5a02")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("054d3b98d7194468a1540d355db87f6d")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("8f459e54d10048a4a4adb661806a8c92")

# Hyper-parameters of unlearned models
print(unlearned_params1, unlearned_params2, unlearned_params3)

cifar100_vit_retrained_metrics, cifar100_vit_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3, percentage=True)
print(f"Retrained: {cifar100_vit_retrained_metrics}")
print(f"Unlearned: {cifar100_vit_unlearned_metrics}")

{'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'lr': '1e-06', 'wd': '0.0005', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 1e-06\n    maximize: False\n    weight_decay: 0.0005\n)', 'tempScheduler': 'exponential', 'minTemp': '1.1', 'maxTemp': '8.0', 'probTransformer': 'gumbel-softmax', 'beta': '0', 'datetime': '09-10-04-05', 'reference_run_name': 'retrained', 'reference_run_id': '7e1d79e5608e4505bf94b34312394f40', 'seed': '3407', 'cudnn': 'slow', 'dataset': 'cifar-100', 'model': 'vit', 'batch_size': '32', 'epochs': '3', 'method': 'our'} {'reference_run_id': '222b19b4fb4a43a4b39b815cefd3913e', 'cudnn': 'slow', 'dataset': 'cifar-100', 'model': 'vit', 'branch_name': 'annealing', 'datetime': '09-10-03-51', 'reference_run_name': 'retrained', 'seed': '13', 'epochs': '3', 'bat

## ImageNet (ViT)

# Class Unlearning

## Cat - CIFAR10 (ResNet18)

In [20]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("16f0a4c834614b5a92c9d7e29fa9c9e1")
retrained_metrics2, retrained_params2 = get_metrics_and_params("6eeb9fe80346484abe64144c248fcff7")
retrained_metrics3, retrained_params3 = get_metrics_and_params("b0cd7a957388400eb9d46aaeb3091ab5")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("a4e36a7079b54c9ca4e7c419c973fde8")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("04ea8a2ebf9344e4b3038b27bf3e2c7e")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("21a7c759854f4fc99cd14a0573b0abf1")

# Hyper-parameters of unlearned models
print(unlearned_params1, unlearned_params2, unlearned_params3)

cat_resnet_retrained_metrics, cat_resnet_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3, percentage=True)
print(f"Retrained: {cat_resnet_retrained_metrics}")
print(f"Unlearned: {cat_resnet_unlearned_metrics}")

{'epochs': '10', 'method': 'our', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'batch_size': '128', 'datetime': '09-10-14-13', 'reference_run_name': 'retrained', 'reference_run_id': '16f0a4c834614b5a92c9d7e29fa9c9e1', 'seed': '3407', 'cudnn': 'slow', 'dataset': 'cifar-10', 'model': 'resnet18', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'lr': '0.0001', 'wd': '0.0005', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0.0005\n)', 'tempScheduler': 'exponential', 'minTemp': '1.1', 'maxTemp': '512.0', 'probTransformer': 'gumbel-softmax', 'beta': '1.0'} {'datetime': '09-10-14-11', 'reference_run_name': 'retrained', 'reference_run_id': '6eeb9fe80346484abe64144c248fcff7', 'seed': '13', 'cudnn': 'slow', 'dataset': 'cifar-10', 'model': 'resnet18', 'batch_size': '128', 'epochs': 

## Horse - CIFAR-10 (ResNet18)

In [25]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("4e22f8ddea6c4251aee1d6560aa0da03")
retrained_metrics2, retrained_params2 = get_metrics_and_params("44b706e806ba49fb8b22dcbe10fc28da")
retrained_metrics3, retrained_params3 = get_metrics_and_params("7935e8838b2240e4a0df1ee2b43ea5d7")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("81341e5d9e3c4c169c8fb3222cb59d32")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("9ca13ccea54747a1bca1090edb73ce1f")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("057b30bbc4dc4816a587d5568104bedc")

# Hyper-parameters of unlearned models
print(unlearned_params1, unlearned_params2, unlearned_params3)

horse_resnet_retrained_metrics, horse_resnet_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3, percentage=True)
print(f"Retrained: {horse_resnet_retrained_metrics}")
print(f"Unlearned: {horse_resnet_unlearned_metrics}")

{'batch_size': '128', 'lr': '0.0001', 'wd': '0.0005', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0.0005\n)', 'reference_run_name': 'retrained', 'reference_run_id': '4e22f8ddea6c4251aee1d6560aa0da03', 'seed': '3407', 'beta': '1.0', 'cudnn': 'slow', 'dataset': 'cifar-10', 'model': 'resnet18', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'datetime': '09-10-15-05', 'epochs': '10', 'method': 'our', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'tempScheduler': 'exponential', 'minTemp': '1.1', 'maxTemp': '512.0', 'probTransformer': 'gumbel-softmax'} {'datetime': '09-10-15-03', 'probTransformer': 'gumbel-softmax', 'dataset': 'cifar-10', 'model': 'resnet18', 'batch_size': '128', 'epochs': '10', 'method': 'our', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'wd': '0.

## Rocket - CIFAR-100 (ResNet18)

In [26]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("9c94c3f3ced7489293b83afc9db646c2")
retrained_metrics2, retrained_params2 = get_metrics_and_params("111fe46ef7e049319f26069c81ef86a6")
retrained_metrics3, retrained_params3 = get_metrics_and_params("356aee9a2da34c82bb5e11fda8b6e137")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("67dbe9379fed40098c1ab66207751c45")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("66804d246f4247069519eefe60e8ec49")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("8123cc5ca8dc46dbb56bf687c1d8af3c")

# Hyper-parameters of unlearned models
print(unlearned_params1, unlearned_params2, unlearned_params3)

rocket_resnet_retrained_metrics, rocket_resnet_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3, percentage=True)
print(f"Retrained: {rocket_resnet_retrained_metrics}")
print(f"Unlearned: {rocket_resnet_unlearned_metrics}")

{'datetime': '09-10-16-13', 'reference_run_name': 'retrained', 'reference_run_id': '03cf6734315b42b98721b1a5c36ae340', 'seed': '3407', 'epochs': '10', 'method': 'our', 'Dr_subset_size': '0.3', 'lr': '0.0001', 'cudnn': 'slow', 'dataset': 'cifar-100', 'model': 'resnet18', 'batch_size': '128', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'branch_name': 'annealing', 'wd': '0.0005', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0.0005\n)', 'tempScheduler': 'exponential', 'minTemp': '1.1', 'maxTemp': '8.0', 'probTransformer': 'gumbel-softmax', 'beta': '1.0'} {'seed': '13', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'reference_run_name': 'retrained', 'reference_run_id': '111fe46ef7e049319f26069c81ef86a6', 'dataset': 'cifar-100', 'model': 'resnet18', 'batch_size'

## Beaver - CIFAR-100 (ResNet18)